In [1]:
from summarizer import Summarizer

In [2]:
import pandas as pd

In [3]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=False)

In [4]:
df = pd.read_csv("../predicted_data_4800_dimension.csv" )
df.drop("Unnamed: 0", inplace=True, axis=1)
df["bert_r1_precision"] = 0.0
df["bert_r2_precision"] = 0.0
df["bert_r1_recall"] = 0.0
df["bert_r2_recall"] = 0.0
df["bert_r1_f1measure"] = 0.0
df["bert_r2_f1measure"] = 0.0
df["skipthoughts_r1_precision"] = 0.0
df["skipthoughts_r2_precision"] = 0.0
df["skipthoughts_r1_recall"] = 0.0
df["skipthoughts_r2_recall"] = 0.0
df["skipthoughts_r1_f1measure"] = 0.0
df["skipthoughts_r2_f1measure"] = 0.0
    


df.head()

,key,location,gt-1,gt-2,gt-3,gt-4,gt-5,pd1,pd2,bert_r1_precision,...,bert_r1_recall,bert_r2_recall,bert_r1_f1measure,bert_r2_f1measure,skipthoughts_r1_precision,skipthoughts_r2_precision,skipthoughts_r1_recall,skipthoughts_r2_recall,skipthoughts_r1_f1measure,skipthoughts_r2_f1measure
0,eyesight-issues_amazon_kindle.txt,/home/arunhiremath/sfsu/nlt/text_summarization...,['Its very soothing for the eyes.'],"['Most users do not encounter eyestrain.\n', '...","['Most users do not encounter eyestrain.\n', '...","['E-ink makes it very easy on the eyes.\n', 'A...","['Very easy on the eyes.\n', 'No fatigue or st...","[""'Easy, on, the, eyes' reading with adjustabl...",None,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,food_holiday_inn_london.txt,/home/arunhiremath/sfsu/nlt/text_summarization...,['The food in the hotel was a little over pric...,"['The food is good, the service great.\n', 'Ve...",['Food was excellent with a wide range of choi...,"['Food can be a little bit overpriced, but is ...",None,"['95 if you want hot food as well .', ', Grea...",None,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,staff_bestwestern_hotel_sfo.txt,/home/arunhiremath/sfsu/nlt/text_summarization...,['The staff was friendly.'],"['Staff is generally friendly, helpful, and ea...","['The staff is friendly and knowledgeable.\n',...",['The staff was very helpful and gave a warm w...,"['Staff are helpful and friendly.\n', 'Staff a...","['The restaurant Pescatore was excellent, frie...",None,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,bathroom_bestwestern_hotel_sfo.txt,/home/arunhiremath/sfsu/nlt/text_summarization...,['People were most impressesd with the bathroo...,['Bathrooms are mostly adequate sized and clea...,"['Bathrooms are satisfactory. \n', 'The rooms ...",['The rooms were not large but were clean and ...,"['Rooms are not big, but clean and with comfor...",['The rooms are exceptionally clean and also t...,None,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,voice_garmin_nuvi_255W_gps.txt,/home/arunhiremath/sfsu/nlt/text_summarization...,"['The voice is a bit robotic.\n', 'The voice i...",['The voice is very clear and loud.'],"['The voices sound robotic.\n', 'TTS mode is t...",['255W garmin gps has more than 750 voices but...,"['Voice is clear and sweet.\n', 'Voice command...",['The most noticeable thing is that the screen...,None,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
fileList = df["location"].tolist()

In [6]:
model = Summarizer()

In [7]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)


In [8]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
def word_length(text):
    return len(tokenizer.tokenize(text))

In [9]:
scores = []

In [16]:
for index,rows in df.iterrows():
    demoFile = open(rows["location"], "rt", encoding="utf8", errors='ignore')
    text = demoFile.read()
    gt1,gt2,gt3,gt4,gt5 = ''.join(rows["gt-1"]),''.join(rows["gt-2"]),''.join(rows["gt-3"]),''.join(rows["gt-4"]),''.join(rows["gt-5"])
    len_original_text =  word_length(text)
    len_summarized_text = word_length(''.join(rows["pd1"]))
    result = model(text,ratio=(len_summarized_text/len_original_text))
    full = ''.join(result)
    df.at[index,"pd2"]  = full
    scores.append(scorer.score(gt1, full))
    scores.append(scorer.score(gt2, full))
    scores.append(scorer.score(gt3, full))
    scores.append(scorer.score(gt4, full))
    scores.append(scorer.score(gt5, full))
    
    #precision
    avg_precision_rogue1 = 0.0
    avg_precision_rogue2 = 0.0
    
    for val in scores:
        avg_precision_rogue1 = val['rouge1'].precision + avg_precision_rogue1
    for val in scores:
        avg_precision_rogue2 = val['rouge2'].precision + avg_precision_rogue2

    
    #recall
    avg_recall_rogue1 = 0.0
    avg_recall_rogue2 = 0.0

    
    for val in scores:
        avg_recall_rogue1 = val['rouge1'].recall + avg_recall_rogue1
    for val in scores:
        avg_recall_rogue2 = val['rouge2'].recall + avg_recall_rogue2
    
    #f1-measure
    avg_fmeasure_rogue1 = 0.0
    avg_fmeasure_rogue2 = 0.0

    for val in scores:
        avg_fmeasure_rogue1 = val['rouge1'].fmeasure + avg_fmeasure_rogue1
    for val in scores:
        avg_fmeasure_rogue2 = val['rouge2'].fmeasure + avg_fmeasure_rogue2
    
    
    df.at[index,"bert_r1_precision"]  = round(avg_precision_rogue1/5,2)
    df.at[index,"bert_r2_precision"] = round(avg_precision_rogue2/5, 2)
    df.at[index,"bert_r1_recall"] = round(avg_recall_rogue1/5, 2)
    df.at[index,"bert_r2_recall"] = round(avg_recall_rogue2/5, 2)
    df.at[index,"bert_r1_f1measure"] = round(avg_fmeasure_rogue1/5, 2)
    df.at[index,"bert_r2_f1measure"] = round(avg_fmeasure_rogue2/5,2)
    print("Length of  Original {} Skipthoughts {}, Length of Bert {}".format(len_original_text,len_summarized_text, word_length(full)))

 

Length of  Original 1624 Skipthoughts 52, Length of Bert 100
Length of  Original 2118 Skipthoughts 18, Length of Bert 28
Length of  Original 5404 Skipthoughts 17, Length of Bert 33
Length of  Original 1458 Skipthoughts 24, Length of Bert 49
Length of  Original 1825 Skipthoughts 59, Length of Bert 67
Length of  Original 1048 Skipthoughts 37, Length of Bert 49
Length of  Original 1808 Skipthoughts 41, Length of Bert 47
Length of  Original 1850 Skipthoughts 53, Length of Bert 89
Length of  Original 3158 Skipthoughts 57, Length of Bert 53
Length of  Original 2319 Skipthoughts 70, Length of Bert 120
Length of  Original 2127 Skipthoughts 36, Length of Bert 59
Length of  Original 3109 Skipthoughts 21, Length of Bert 58
Length of  Original 1235 Skipthoughts 73, Length of Bert 135
Length of  Original 1069 Skipthoughts 48, Length of Bert 78
Length of  Original 1260 Skipthoughts 24, Length of Bert 52
Length of  Original 1472 Skipthoughts 15, Length of Bert 31
Length of  Original 1980 Skipthoughts

In [29]:
scores = []
for index,rows in df.iterrows():
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=False)
    gt1,gt2,gt3,gt4,gt5 = ''.join(rows["gt-1"]),''.join(rows["gt-2"]),''.join(rows["gt-3"]),''.join(rows["gt-4"]),''.join(rows["gt-5"])
    result = ''.join(rows["pd2"])
    scores.append( scorer.score(gt1, result))
    scores.append( scorer.score(gt2, result))
    scores.append( scorer.score(gt3, result))
    scores.append( scorer.score(gt4, result))
    scores.append( scorer.score(gt5, result))

    #precision
    avg_precision_rogue1 = 0.0
    avg_precision_rogue2 = 0.0
    
    for val in scores:
        avg_precision_rogue1 = val['rouge1'].precision + avg_precision_rogue1
    for val in scores:
        avg_precision_rogue2 = val['rouge2'].precision + avg_precision_rogue2

    
    #recall
    avg_recall_rogue1 = 0.0
    avg_recall_rogue2 = 0.0

    
    for val in scores:
        avg_recall_rogue1 = val['rouge1'].recall + avg_recall_rogue1
    for val in scores:
        avg_recall_rogue2 = val['rouge2'].recall + avg_recall_rogue2
    
    #f1-measure
    avg_fmeasure_rogue1 = 0.0
    avg_fmeasure_rogue2 = 0.0

    for val in scores:
        avg_fmeasure_rogue1 = val['rouge1'].fmeasure + avg_fmeasure_rogue1
    for val in scores:
        avg_fmeasure_rogue2 = val['rouge2'].fmeasure + avg_fmeasure_rogue2
        
    df.at[index,"bert_r1_precision"]  = round(avg_precision_rogue1/5,2)
    df.at[index,"bert_r2_precision"] = round(avg_precision_rogue2/5, 2)
    df.at[index,"bert_r1_recall"] = round(avg_recall_rogue1/5, 2)
    df.at[index,"bert_r2_recall"] = round(avg_recall_rogue2/5, 2)
    df.at[index,"bert_r1_f1measure"] = round(avg_fmeasure_rogue1/5, 2)
    df.at[index,"bert_r2_f1measure"] = round(avg_fmeasure_rogue2/5,2)
    


KeyError: 'rouge2'

In [30]:
len("Hello World")

11

In [25]:
#rogue, precision and f1 measure for the pd1
scores = []

In [26]:
scores = []
for index,rows in df.iterrows():
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=False)
    gt1,gt2,gt3,gt4,gt5 = ''.join(rows["gt-1"]),''.join(rows["gt-2"]),''.join(rows["gt-3"]),''.join(rows["gt-4"]),''.join(rows["gt-5"])
    result = ''.join(rows["pd1"])
    scores.append( scorer.score(gt1, result))
    scores.append( scorer.score(gt2, result))
    scores.append( scorer.score(gt3, result))
    scores.append( scorer.score(gt4, result))
    scores.append( scorer.score(gt5, result))

    #precision
    avg_precision_rogue1 = 0.0
    avg_precision_rogue2 = 0.0
    
    for val in scores:
        avg_precision_rogue1 = val['rouge1'].precision + avg_precision_rogue1
    for val in scores:
        avg_precision_rogue2 = val['rouge2'].precision + avg_precision_rogue2

    
    #recall
    avg_recall_rogue1 = 0.0
    avg_recall_rogue2 = 0.0

    
    for val in scores:
        avg_recall_rogue1 = val['rouge1'].recall + avg_recall_rogue1
    for val in scores:
        avg_recall_rogue2 = val['rouge2'].recall + avg_recall_rogue2
    
    #f1-measure
    avg_fmeasure_rogue1 = 0.0
    avg_fmeasure_rogue2 = 0.0

    for val in scores:
        avg_fmeasure_rogue1 = val['rouge1'].fmeasure + avg_fmeasure_rogue1
    for val in scores:
        avg_fmeasure_rogue2 = val['rouge2'].fmeasure + avg_fmeasure_rogue2
        
    df.at[index,"skipthoughts_r1_precision"]  = round(avg_precision_rogue1/5,2)
    df.at[index,"skipthoughts_r2_precision"] = round(avg_precision_rogue2/5, 2)
    df.at[index,"skipthoughts_r1_recall"] = round(avg_recall_rogue1/5, 2)
    df.at[index,"skipthoughts_r2_recall"] = round(avg_recall_rogue2/5, 2)
    df.at[index,"skipthoughts_r1_f1measure"] = round(avg_fmeasure_rogue1/5, 2)
    df.at[index,"skipthoughts_r2_f1measure"] = round(avg_fmeasure_rogue2/5,2)
    


In [27]:
#save the file to csv
df.to_csv("complete_analysis.csv")

In [28]:
df.head()

,key,location,gt-1,gt-2,gt-3,gt-4,gt-5,pd1,pd2,bert_r1_precision,...,bert_r1_recall,bert_r2_recall,bert_r1_f1measure,bert_r2_f1measure,skipthoughts_r1_precision,skipthoughts_r2_precision,skipthoughts_r1_recall,skipthoughts_r2_recall,skipthoughts_r1_f1measure,skipthoughts_r2_f1measure
0,eyesight-issues_amazon_kindle.txt,/home/arunhiremath/sfsu/nlt/text_summarization...,['Its very soothing for the eyes.'],"['Most users do not encounter eyestrain.\n', '...","['Most users do not encounter eyestrain.\n', '...","['E-ink makes it very easy on the eyes.\n', 'A...","['Very easy on the eyes.\n', 'No fatigue or st...","[""'Easy, on, the, eyes' reading with adjustabl...",It feels as easy to read as the K1 but doesn't...,0.08,...,0.44,0.02,0.13,0.01,0.13,0.04,0.38,0.13,0.19,0.05
1,food_holiday_inn_london.txt,/home/arunhiremath/sfsu/nlt/text_summarization...,['The food in the hotel was a little over pric...,"['The food is good, the service great.\n', 'Ve...",['Food was excellent with a wide range of choi...,"['Food can be a little bit overpriced, but is ...",None,"['95 if you want hot food as well .', ', Grea...",The room was packed to capacity with queues at...,0.22,...,0.65,0.08,0.30,0.05,0.33,0.12,0.60,0.22,0.39,0.14
2,staff_bestwestern_hotel_sfo.txt,/home/arunhiremath/sfsu/nlt/text_summarization...,['The staff was friendly.'],"['Staff is generally friendly, helpful, and ea...","['The staff is friendly and knowledgeable.\n',...",['The staff was very helpful and gave a warm w...,"['Staff are helpful and friendly.\n', 'Staff a...","['The restaurant Pescatore was excellent, frie...",The staff in the morning seemed to need anothe...,0.35,...,1.05,0.22,0.48,0.10,0.57,0.12,1.01,0.22,0.67,0.14
3,bathroom_bestwestern_hotel_sfo.txt,/home/arunhiremath/sfsu/nlt/text_summarization...,['People were most impressesd with the bathroo...,['Bathrooms are mostly adequate sized and clea...,"['Bathrooms are satisfactory. \n', 'The rooms ...",['The rooms were not large but were clean and ...,"['Rooms are not big, but clean and with comfor...",['The rooms are exceptionally clean and also t...,"The room was not overly big, but clean and ver...",0.53,...,1.45,0.35,0.73,0.17,0.90,0.23,1.37,0.34,1.01,0.26
4,voice_garmin_nuvi_255W_gps.txt,/home/arunhiremath/sfsu/nlt/text_summarization...,"['The voice is a bit robotic.\n', 'The voice i...",['The voice is very clear and loud.'],"['The voices sound robotic.\n', 'TTS mode is t...",['255W garmin gps has more than 750 voices but...,"['Voice is clear and sweet.\n', 'Voice command...",['The most noticeable thing is that the screen...,The voice prompts and maps are wonderful espec...,0.62,...,1.95,0.64,0.87,0.24,0.98,0.25,1.79,0.44,1.15,0.29
